In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# ruta de los .csv
ruta_input = '/kaggle/input'

for dirname, _, filenames in os.walk(ruta_input):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [5]:
'''
PassengerId - Identificación del Pasajero
Pclass - Clase del Pasajero
Name - Nombre del Pasajero
Sex - Género del Pasajero
Age - Edad del Pasajero
SibSp - Número de Hermanos/Cónyuges a Bordo (Siblings/Spouses)
Parch - Número de Padres/Hijos a Bordo (Parents/Children)
Ticket - Número de Boleto
Fare - Tarifa Pagada por el Pasajero
Cabin - Cabina del Pasajero
Embarked - Puerto de Embarque del Pasajero
'''

ruta_test = "/kaggle/input/titanic/test.csv"
ruta_train = "/kaggle/input/titanic/train.csv"

# importa data de test.csv y train.csv: esto son los dataframes
test_data = pd.read_csv(ruta_test)
train_data = pd.read_csv(ruta_train)

# muestrame los 5 primeros registros
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
# Explorando un patron: 
# --> Las mujeres viven 
# --> Los hombres mueren

'''
train_data: dataframe (DF)
train_data.Sex == 'female': crea una serie booleana, 
                            si la columna Sex = female devuelve True
train_data.loc[]: filtra filas del DF basandose en la condicion de 
                  dentro de los corchetes
["Survived"]: se coloca despues de train_data.loc[] para especificar 
              que columna queremos seleccionar del DF resultante
              en este caso solo seleccionamos la columna "Survived" 
              de las filas que cumplan el genero femenino
'''


print('\ntrain_data.Sex == \'female\'')
print(train_data.Sex == 'female')

print('\ntrain_data.loc[train_data.Sex == \'female\']')
print(train_data.loc[train_data.Sex == 'female'])

print('\ntrain_data.loc[train_data.Sex == \'female\'][\"Survived\"]')
print(train_data.loc[train_data.Sex == 'female']["Survived"])

women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print('\nPorcentajes: ')
print("% of women who survived:", rate_women)
print("% of men who survived:", rate_men)


'''
Since gender seems to be such a strong indicator of survival, 
the submission file in gender_submission.csv is not a bad first guess!

But at the end of the day, this gender-based submission bases its 
predictions on only a single column. As you can imagine, by considering 
multiple columns, we can discover more complex patterns

Since it is quite difficult to consider several columns at once 
(or, it would take a long time to consider all possible patterns in 
many different columns simultaneously), we'll use machine learning to 
automate this for us.
'''


train_data.Sex == 'female'
0      False
1       True
2       True
3       True
4      False
       ...  
886    False
887     True
888     True
889    False
890    False
Name: Sex, Length: 891, dtype: bool

train_data.loc[train_data.Sex == 'female']
     PassengerId  Survived  Pclass  \
1              2         1       1   
2              3         1       3   
3              4         1       1   
8              9         1       3   
9             10         1       2   
..           ...       ...     ...   
880          881         1       2   
882          883         0       3   
885          886         0       3   
887          888         1       1   
888          889         0       3   

                                                  Name     Sex   Age  SibSp  \
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel) 

"\nSince gender seems to be such a strong indicator of survival, \nthe submission file in gender_submission.csv is not a bad first guess!\n\nBut at the end of the day, this gender-based submission bases its \npredictions on only a single column. As you can imagine, by considering \nmultiple columns, we can discover more complex patterns\n\nSince it is quite difficult to consider several columns at once \n(or, it would take a long time to consider all possible patterns in \nmany different columns simultaneously), we'll use machine learning to \nautomate this for us.\n"

In [45]:
# Ejemplo Matriz caracteristica

# diccionario example_data
example_data = {
    "Pclass": [1, 2, 3, 1, 2],
    "Sex": ["Male", "Female", "Female", "Male", "Female"]
}

# Creando el DF a partir del diccionario
example_train_data = pd.DataFrame(example_data)

example_features = ["Pclass", "Sex"]
example_X = pd.get_dummies(example_train_data[example_features])

print(example_X)

   Pclass  Sex_Female  Sex_Male
0       1       False      True
1       2        True     False
2       3        True     False
3       1       False      True
4       2        True     False


In [9]:
# Your 1st Machine Learning (ML) model: Random Forest Model
# Formación\Ejercicios\ML & DL\images


# importamos la libreria RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

# y: variable objetivo a predecir
# Contiene etiquetas de supervivencia del conjunto de datos de entrenamiento
# se asume que train data contiene la columna ["Survived"]
y = train_data["Survived"]
print('\ntrain_data[\"Survived\"]')
print(y)


# features: seleccion de las caracteristicas para entrenar el modelo
features = ["Pclass", "Sex", "SibSp", "Parch"]

# Matrices de caracteristicas de train y test a traves de pandas
# Convierte las variables categoricas a variables numericas
X_train = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

print('\nMatriz caracteristica X_train')
print(X_train)
print('\nMatriz caracteristica X_test')
print(X_test)


# Creando el modelo RandomForestClassifier
'''
--> n_estimators=100: nº de arboles de decision que se crearan en el bosques (100)
--> max_depth=5: profundidad de cada arbol
                 si tiene mucha profundidad hay un sobreajuste (no generaliza bien)
--> random_state=1: semilla para la generacion de nº pseudoaleatorios
                    garantiza que los resultados sean reproducibles.
                    Fija un punto de inicio para la generacion de numeros pseudoaleatorios.
                    --> Reproducibilidad: 
                        Misma semilla, mismos resultados
                    --> Consistencia: 
                        Todos debemos utilizar la misma semilla, mismos resultados
                    --> Depuracion y ajustes de hiperparametros:
                        Hiperparametros: configs antes de iniciar el proceso de entrenamiento
                        Al ajustar el modelo es util que las diferencias de rendimiento no se
                        deban a variaciones aleatorias, sino a cambios en hiperparametros o datos.
                        La semilla asegura que la variabilidad debido a la aleatoriedad sea controlada
'''
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)

# Entrenando el modelo con los datos de entrenamiento
# model.fit(matriz_caracteristica, variable_objetivo_a_predecir)
model.fit(X_train, y)

# A partir del modelo, con la matriz caracteristica X_test hazme las predicciones
predictions = model.predict(X_test)

# Crear un DataFrame llamado output con 2 columnas: PassengerId y Survived
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})

# Guardar el output en un csv
output.to_csv('RandomForestClassifierSubmision.csv', index=False)

# Si llega hasta aqui todo deberia de haber funcionado
# index=True crearia una columna adicional con ids autoincrementales
print("Your submission was successfully saved!")


train_data["Survived"]
0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

Matriz caracteristica X_train
     Pclass  SibSp  Parch  Sex_female  Sex_male
0         3      1      0       False      True
1         1      1      0        True     False
2         3      0      0        True     False
3         1      1      0        True     False
4         3      0      0       False      True
..      ...    ...    ...         ...       ...
886       2      0      0       False      True
887       1      0      0        True     False
888       3      1      2        True     False
889       1      0      0       False      True
890       3      0      0       False      True

[891 rows x 5 columns]

Matriz caracteristica X_test
     Pclass  SibSp  Parch  Sex_female  Sex_male
0         3      0      0       False      True
1         3      1      0        True     False
2         2      0      0     